In [43]:
# Data visualization dependencies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

# Data analytics libraries
import numpy as np
import pandas as pd

# other dependencies
import datetime as dt
from dateutil.relativedelta import relativedelta

# Reflect Tables into SQLAlchemy ORM

In [28]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.sql import select

In [3]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [6]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [7]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [155]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database
# query for the latest date
latest_date_max = session.query(func.max(Measurement.date)).scalar()
# convert the latest date to a datetime object
latest_date = dt.datetime.strptime(latest_date_max, '%Y-%m-%d').date()
# use timedelta to find the date 1 year ago
year_ago = latest_date - dt.timedelta(days=365)

# Create SQL query that will store the dates between year_ago and latest_date 
from sqlalchemy import between
precipitation_query = select([Measurement.date, Measurement.prcp]).\
where(between(Measurement.date, year_ago,latest_date))

# Store the query values into a data frame and set the index as the date
conn = engine.connect()
precipitation_df = pd.read_sql(precipitation_query, conn)
precipitation_df = precipitation_df.set_index('date')

In [161]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_summary_df = precipitation_df.describe()

# Output to JN window
precipitation_summary_df

,prcp
count,2021.000000
mean,0.177279
std,0.461190
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


In [172]:
# Count the number of unique stations
number_of_stations = session.query(Measurement.station).group_by(Measurement.station).count()

print(f"There are {number_of_stations} stations in the dataset.")

There are 9 stations in the dataset.


In [187]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

# Design a query to show how many stations are available in this dataset?
station_query = select([Measurement.station, func.count(Measurement.station)])\
.group_by(Measurement.station).order_by(func.count(Measurement.station).desc())

# Store the query in a dataframe and output to JN
station_count_df = pd.read_sql(station_query, conn)
station_count_df = station_count_df.rename(columns = {"count_1" : "count"})

# output to JN
station_count_df

,station,count
0,USC00519281,2772
1,USC00519397,2724
2,USC00513117,2709
3,USC00519523,2669
4,USC00516128,2612
5,USC00514830,2202
6,USC00511918,1979
7,USC00517948,1372
8,USC00518838,511


In [200]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?

# get the station tag of the station with the most amount of records
station_tag = station_count_df.iloc[1,0]

# Query for all the temperature data
station_query = select([Measurement.tobs]).where(Measurement.station == station_tag)

max_station_temp_df = pd.read_sql(station_query,conn)
max_temp = max(max_station_temp_df["tobs"])
min_temp = min(max_station_temp_df["tobs"])
avg_temp = round(max_station_temp_df["tobs"].mean(), 1)

# print stats to JN
print(f"MAX, MIN, and AVG temperature for {station_tag} station:")
print(f"MAX: {max_temp}")
print(f"MIN: {min_temp}")
print(f"AVG: {avg_temp}")

MAX, MIN, and AVG temperature for USC00519397 station:
MAX: 87.0
MIN: 56.0
AVG: 74.6


In [224]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
from sqlalchemy import and_

# create a query 
temperature_query = select([Measurement.date, Measurement.tobs]).\
where(and_(Measurement.station == station_tag, between(Measurement.date, year_ago,latest_date)))

# 
top_station_temps_df = pd.read_sql(temperature_query, conn)

# print to JN
top_station_temps_df.head()

,date,tobs
0,2016-08-23,81.0
1,2016-08-24,79.0
2,2016-08-25,80.0
3,2016-08-26,79.0
4,2016-08-27,77.0


In [210]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]


In [223]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

# 
date_2016 = latest_date - dt.timedelta(days=365)
year_ago_from_date_2016 = year_ago - dt.timedelta(days=365)

# 
previous_year_stats = calc_temps(year_ago_from_date_2016,date_2016)

# Print to JN
print(f"MAX, MIN, and AVG temperature for {station_tag} station between 2015-2016:")
print(f"MAX: {previous_year_stats[0]}")
print(f"MIN: {previous_year_stats[1]}")
print(f"AVG: {previous_year_stats[2]}")

MAX, MIN, and AVG temperature for USC00519397 station between 2015-2016:
MAX: (56.0, 74.43899317406144, 86.0)


IndexError: list index out of range

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
